In [1]:
from pymongo import MongoClient, GEOSPHERE

from pprint import pprint
from datetime import datetime

In [2]:
username = "vt2182"
password = "vt2182"
host = "mongo-csgy-6513-spring.db"
auth_database = "vt2182"

In [3]:
client = MongoClient(f"mongodb://{username}:{password}@{host}/{auth_database}")

In [4]:
db = client[auth_database]
db

Database(MongoClient(host=['mongo-csgy-6513-spring.db:27017'], document_class=dict, tz_aware=False, connect=True), 'vt2182')

In [5]:
db.list_collection_names()

['worldcities',
 'durham_restaurants',
 'meteorites',
 'durham_foreclosures',
 'restaurants']

## Q2. Restaurant foreclosures in North Carolina

Some Background: geospatial logic is possible in MongoDB using the geopastial library/facilities. https://docs.mongodb.com/manual/geospatial-queries/

Datasets:
- `durham-nc-foreclosure-2006-2016.json`
- `Restaurants_in_Durham_County_NC.csv`

Find the number of foreclosures within the _polygon_ defined as all restaurants where `Rpt_Area_Desc=”Food Service”` and `Seats>=100`.

In [6]:
foreclosures = db["durham_foreclosures"]
foreclosures.count_documents({})

1948

In [7]:
pprint(foreclosures.find_one())

{'_id': ObjectId('64447e0248ae79980f06c9f9'),
 'datasetid': 'foreclosure-2006-2016',
 'fields': {'address': '217 E CORPORATION ST',
            'geocode': [36.0013755, -78.8922549],
            'parcel_number': '110138',
            'year': '2006'},
 'geometry': {'coordinates': [-78.8922549, 36.0013755], 'type': 'Point'},
 'record_timestamp': '2017-03-06T12:41:48-05:00',
 'recordid': '629979c85b1cc68c1d4ee8cc351050bfe3592c62'}


In [8]:
restaurants = db["durham_restaurants"]
restaurants.count_documents({})

4944

In [9]:
pprint(restaurants.find_one())

{'Closing_Date': '',
 'Est_Group_Desc': 'Nursing Home',
 'Hours_Of_Operation': '',
 'ID': 58123,
 'Insp_Freq': 4,
 'Opening_Date': '2003-10-15',
 'Premise_Address1': '4434 BEN FRANKLIN BLVD',
 'Premise_Address2': '',
 'Premise_City': 'DURHAM',
 'Premise_Name': 'BROOKDALE DURHAM IFS',
 'Premise_Phone': '(919) 479-9966',
 'Premise_State': 'NC',
 'Premise_Zip': 27704,
 'Risk': 4,
 'Rpt_Area_Desc': 'Food Service',
 'Seats': 350,
 'Sewage': '3 - Municipal/Community',
 'Smoking_Allowed': 'NO',
 'Status': 'ACTIVE',
 'Transitional_Type_Desc': 'FOOD',
 'Type_Description': '16 - Institutional Food Service',
 'Water': '5 - Municipal/Community',
 '_id': ObjectId('64447cea904e19136eb23ad9'),
 'geolocation': '36.0467802, -78.8895483'}


In [10]:
# Create a 2dsphere index for the 'geometry' field if it doesn't exist
foreclosures.create_index([("geometry", GEOSPHERE)], background=True)

'geometry_2dsphere'

In [11]:
pprint(foreclosures.find_one())

{'_id': ObjectId('64447e0248ae79980f06c9f9'),
 'datasetid': 'foreclosure-2006-2016',
 'fields': {'address': '217 E CORPORATION ST',
            'geocode': [36.0013755, -78.8922549],
            'parcel_number': '110138',
            'year': '2006'},
 'geometry': {'coordinates': [-78.8922549, 36.0013755], 'type': 'Point'},
 'record_timestamp': '2017-03-06T12:41:48-05:00',
 'recordid': '629979c85b1cc68c1d4ee8cc351050bfe3592c62'}


In [12]:
# Filter the restaurant dataset based on the conditions
filtered_restaurants = list(restaurants.find({"Rpt_Area_Desc": "Food Service", "Seats": {"$gte": 100}}))
len(filtered_restaurants)

464

In [13]:
pprint(filtered_restaurants[0])

{'Closing_Date': '',
 'Est_Group_Desc': 'Nursing Home',
 'Hours_Of_Operation': '',
 'ID': 58123,
 'Insp_Freq': 4,
 'Opening_Date': '2003-10-15',
 'Premise_Address1': '4434 BEN FRANKLIN BLVD',
 'Premise_Address2': '',
 'Premise_City': 'DURHAM',
 'Premise_Name': 'BROOKDALE DURHAM IFS',
 'Premise_Phone': '(919) 479-9966',
 'Premise_State': 'NC',
 'Premise_Zip': 27704,
 'Risk': 4,
 'Rpt_Area_Desc': 'Food Service',
 'Seats': 350,
 'Sewage': '3 - Municipal/Community',
 'Smoking_Allowed': 'NO',
 'Status': 'ACTIVE',
 'Transitional_Type_Desc': 'FOOD',
 'Type_Description': '16 - Institutional Food Service',
 'Water': '5 - Municipal/Community',
 '_id': ObjectId('64447cea904e19136eb23ad9'),
 'geolocation': '36.0467802, -78.8895483'}


In [14]:
pprint(filtered_restaurants[0]['geolocation'])

'36.0467802, -78.8895483'


In [15]:
polygon_coords = []

for row in filtered_restaurants:
    if row['geolocation']:
        latitude, longitude = map(float, row['geolocation'].split(', '))
    else:
        continue
    
    polygon_coords.append([longitude, latitude])

In [16]:
lonMin = min(coord[0] for coord in polygon_coords)
lonMax = max(coord[0] for coord in polygon_coords)
latMin = min(coord[1] for coord in polygon_coords)
latMax = max(coord[1] for coord in polygon_coords)

In [17]:
# number of foreclosures within the polygon defined as all restaurants 
# where `Rpt_Area_Desc=”Food Service”` and `Seats>=100`

foreclosures_within_polygon = foreclosures.find({
    "geometry":{
        "$geoWithin":{
            "$geometry":{
                "type":"Polygon",
                "coordinates":[[
                    [lonMin,latMin],
                    [lonMin,latMax],
                    [lonMax,latMax],
                    [lonMax,latMin],
                    [lonMin,latMin]
                ]]
            }
        }
    }
})

count=0
for foreclosure in foreclosures_within_polygon:
    count += 1

print("Number of foreclosures within the polygon:", count)

Number of foreclosures within the polygon: 1830


# Q3. Extra Credit

Datasets:
- `meteorites.json`
- `worldcities.csv`

Use the MongoDB geospatial facilities to *find the nearest city to each meteorite “fallen” (not found) since the year 1950, inclusive*. Distance is between coordinates should be a straight line. 

Note: 
- `worldcities` is a CSV file. You will need to import into MongoDB AND clean-up the double quotes.
- Use the `$near` operator and select this closest entry per city.


In [18]:
meteorites = db["meteorites"]
meteorites.count_documents({})

2000

In [19]:
pprint(meteorites.find_one())

{'_id': ObjectId('6444776e8eccd146808a65c1'),
 'fall': 'Fell',
 'geolocation': {'coordinates': [10.23333, 56.18333], 'type': 'Point'},
 'id': '2',
 'mass': '720',
 'name': 'Aarhus',
 'nametype': 'Valid',
 'recclass': 'H6',
 'reclat': '56.183330',
 'reclong': '10.233330',
 'year': '1951-01-01T00:00:00.000'}


In [20]:
worldcities = db["worldcities"]
worldcities.count_documents({})

25786

In [21]:
# Add city_location field as a GeoJSON Point
worldcities.update_many(
    {},
    [
        {
            "$set": {
                "city_location": {
                    "type": "Point",
                    "coordinates": ["$lng", "$lat"]
                }
            }
        }
    ]
)

In [22]:
pprint(worldcities.find_one())

{'_id': ObjectId('6444778e99954515fc972d86'),
 'admin_name': 'Malishevë',
 'capital': 'admin',
 'city': 'Malishevë',
 'city_ascii': 'Malisheve',
 'city_location': {'coordinates': [20.7458, 42.4822], 'type': 'Point'},
 'country': 'Kosovo',
 'id': 1901597212,
 'iso2': 'XK',
 'iso3': 'XKS',
 'lat': 42.4822,
 'lng': 20.7458,
 'population': ''}


In [23]:
# Create 2dsphere indexes
meteorites.create_index([('geolocation', '2dsphere')])
worldcities.create_index([('city_location', '2dsphere')])

'city_location_2dsphere'

In [24]:
# find the nearest city to each meteorite “fallen” (not found) since the year 1950, inclusive

filtered_meteorites = meteorites.find({
    "fall": "Fell",
    "year": {
        "$gte": datetime.strptime('1950, 1, 1', '%Y, %m, %d').isoformat(timespec='milliseconds')
    },
    "geolocation": {
        "$exists": True
    }
})

for meteorite in filtered_meteorites:
    nearest_city = worldcities.find_one({
        "city_location": {
            "$near": {
                "$geometry": {
                    "type": "Point",
                    "coordinates": meteorite["geolocation"]["coordinates"],
                },
            },
        },
    })

    print("meteorite {} and nearest city {}".format(meteorite['name'], nearest_city["city"]))

meteorite Aarhus and nearest city Århus
meteorite Abee and nearest city Athabasca
meteorite Acapulco and nearest city Acapulco
meteorite Aioun el Atrouss and nearest city Ayoun el Atrous
meteorite Akwanga and nearest city Lafia
meteorite Akyumak and nearest city Ağrı
meteorite Al Rais and nearest city Medina
meteorite Al Zarnkh and nearest city En Nuhud
meteorite Alby sur Chéran and nearest city Annecy
meteorite Allende and nearest city Hidalgo del Parral
meteorite Almahata Sitta and nearest city Wadi Halfa
meteorite Alta'ameem and nearest city Kirkuk
meteorite Andreevka and nearest city Kramatorsk
meteorite Anlong and nearest city Xingyi
meteorite Aomori and nearest city Aomori
meteorite Arbol Solo and nearest city San Luis
meteorite Arroyo Aguiar and nearest city Santa Fe
meteorite Ash Creek and nearest city Lacy-Lakeview
meteorite Avanhandava and nearest city Penápolis
meteorite Awere and nearest city Kole
meteorite Ban Rong Du and nearest city Phetchabun
meteorite Barwell and neare